# <center> GEC
GEC pour tester les modèles et les métriques en sortie.
Implémentation de la métrique GLEU ?
Utilisation du modèle BART ? Ou de l'implémentation de BERT ? Ou de l'implémentation de GPT-2 ? Ou de l'implémentation de T5 ? Ou de l'implémentation de GPT-3 ?

Implémentation d'un modèle GEC en simulation de la pipeline.


Modèle T5:
https://huggingface.co/vennify/t5-base-grammar-correction

Alors, de ce que j'ai vu on peut utiliser des modèles text to text comme le modèle transformer T5 de google ou le modèle BART de facebook.
BART : https://ai.meta.com/research/publications/bart-denoising-sequence-to-sequence-pre-training-for-natural-language-generation-translation-and-comprehension/
T5 : https://blog.research.google/2020/02/exploring-transfer-learning-with-t5.html
En se référant au SOTA de la tâche GEC : on peut considérer les modèles transformer comme étant les meilleurs modèles pour la tâche GEC.
https://arxiv.org/pdf/2106.03830.pdf
https://aclanthology.org/2022.coling-1.246/
https://dl.acm.org/doi/abs/10.1145/3474840

https://scholar.google.com/scholar?hl=fr&as_sdt=0%2C5&q=gec+grammatical+error+correction&btnG=&oq=GEC

Aussi différents concours on eu lieu, comme le BEA :
en 2019 le BEA aborde la tâche GEC. https://www.cl.cam.ac.uk/research/nl/bea2019st/
**https://competitions.codalab.org/competitions/20228#results**
https://aclanthology.org/W19-4406.pdf

SOTA 2020 : grammarly avec GECTor : https://arxiv.org/pdf/2005.12592v2.pdf   https://github.com/grammarly/gector
best SOTA ? : https://aclanthology.org/2022.naacl-main.143/


huggingface pretrained : https://huggingface.co/transformers/v3.3.1/pretrained_models.html

https://huggingface.co/docs/transformers/model_doc/t5

https://huggingface.co/docs/transformers/model_doc/bart
papiers bart : https://arxiv.org/abs/2005.11849

BTS TTT :  https://aclanthology.org/2021.wat-1.10.pdf

https://aclanthology.org/W19-4413.pdf
LLM GEC : https://aclanthology.org/2021.emnlp-main.611.pdf


BERT
https://sunilchomal.github.io/GECwBERT/
https://huggingface.co/bert-base-uncased


GPT3 https://aclanthology.org/2023.bea-1.18.pdf
https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo
https://arxiv.org/abs/2303.13648


GEC VIDEO (BANGER0): https://aclanthology.org/2022.naacl-main.143.mp4

Evaluation avec F0.5, GLUE, BLUE, ROUGE, exact match...

___
## Imports libraries

In [3]:
from datasets import load_dataset
import torch, os
from torch.utils.data import DataLoader, Dataset
from datetime import date

from tqdm.notebook import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from happytransformer import HappyTextToText, TTSettings
from evaluate import load
#!pip3 install bert-score
bertscore = load("bertscore")

___
## Import des données

In [17]:
class ASR_Dataset(Dataset):
    def __init__(self, path, text=None):
        
        self.sentences = []
        
        if text is not None:
            self.sentences.append(text.strip())
        else:
            self.path = path
            self.files = os.listdir(path)
            for file in self.files:
                with open(path + '\\' + file, 'r', encoding='utf-8') as file:
                    for line in file:
                        self.sentences.append(line.strip())

    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        transcript = self.sentences[idx]
        return transcript


def load_jfleg_dataset(path='data\\', text=None):
    jfleg_dataset = load_dataset("jfleg", "test", split="test", cache_dir=path)
    jfleg_Dataloader = DataLoader(jfleg_dataset, batch_size=1)
    
    print("Number of samples:", len(jfleg_dataset))

    sample_meta0 = jfleg_Dataloader.dataset[0]
    print("""
    Transcript n°0 :

    Path audio: {}
    Sentence: {}
    Corrections: {}
    """.format(path, sample_meta0['sentence'], sample_meta0['corrections']))
    
    return jfleg_Dataloader

def process_jfleg(data, model='T5'):
    sentence = data['sentence']
    corrections = data['corrections']
    
    return sentence[0], 0, corrections


def load_ASR_dataset(path='data\\ASR', text=None, dataset=ASR_Dataset):
    ASR_Dataset = dataset(path, text)
    ASR_Dataloader = DataLoader(ASR_Dataset, batch_size=1)
    
    print("Number of samples:", len(ASR_Dataset))
    
    sample_meta0 = ASR_Dataset
    print("""
    Transcript n°0 :
    
    Transcript : {}
    """.format(*sample_meta0))
    
    return ASR_Dataloader

def process_ASR(data, model='T5'):
    return data[0], 0, None

def evaluate_jfleg(input, labels, model, tokenizer):
    results = []
    bert_scores = []
    for correction in labels:
        if load('exact_match').compute(references=correction, predictions=[input])['exact_match'] == 1:
            return correction, 5, None
        
        bert_scores.append(bertscore.compute(predictions=[input], references=correction, lang="en"))
        input_ids = tokenizer.encode("stsb sentence 1: "+input+" sentence 2: "+correction[0], return_tensors="pt").to("cuda")
        stsb_ids = model.generate(input_ids)
        stsb = tokenizer.decode(stsb_ids[0],skip_special_tokens=True)
        results.append(float(stsb))
    
    max_arg = results.index(max(results))
    max_score = results[max_arg]
    
    max_bert = {}
    for scores in bert_scores:
        for key, value in scores.items():
            if key != "hashcode":
                if key not in max_bert:
                    max_bert[key] = value
                else:
                    max_bert[key] = max(max_bert[key], value)
    
    return labels[max_arg], max_score, max_bert
    

In [18]:
def GEC(text:str = None, model='T5', output_folder='output', dataset='ASR'):
    pipeline = "/pipeline/" if text is not None else "/"
    correction_folder = output_folder + pipeline + model + '_' + dataset
    correction_file =f'{correction_folder}/output_GEC.{date.today()}.txt'
    verification = f'{correction_folder}/Verification.{date.today()}.txt'
    vard = f'{correction_folder}/vard.{date.today()}.txt'
    os.makedirs(correction_folder, exist_ok=True)
    
    #TQDM loader
    try:
        function = "load_" + dataset + "_dataset"
        dataloader = eval(function)(text=text)
        dataloader_tqdm = tqdm(dataloader, total=len(dataloader))
        
        process_function = "process_" + dataset
    except Exception as e:
        raise ValueError(f'Unknown dataset {dataset}{e}')
    
    print(f'Generating corrections for {dataset} with {model} in {correction_folder}')
    if model == 'T5':
        T5 = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
        args = TTSettings(num_beams=5, min_length=1, max_length=100)
    
    with open(correction_file, 'w', encoding='utf-8') as file:
        file.write('')
        
    if dataset == 'jfleg':
        with open(verification, 'w', encoding='utf-8') as file:
                    file.write('')
        with open(vard, 'w', encoding='utf-8') as file:
                    file.write('')
 
    tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=512)
    model_evaluation = T5ForConditionalGeneration.from_pretrained('t5-base').to("cuda")
    mean_evaluation_score = 0
    mean_bert = {}
    result = ""
    for i, data in enumerate(dataloader_tqdm):
        (sentence, out, extra) = eval(process_function)(data, model=model)
        
        if out == 1:
            dataloader_tqdm.set_postfix({'status': 'Skipped', 'ID': i})
            continue
            
        try:
            #Passage du transcript dans le modèle
            if model == 'T5':
                result = T5.generate_text("grammar:" + sentence, args=args).text
                with open(correction_file, 'a', encoding='utf-8') as file:
                    file.write(result + '\n')
                if dataset == 'jfleg':
                    with open(vard, 'a', encoding='utf-8') as file:
                        file.write(sentence + '\n')
            elif model == 'BART':
                result = ''
            else:
                raise ValueError(f'Unknown model {model}')
            
            dataloader_tqdm.set_postfix()

        except Exception as e:
            dataloader_tqdm.set_postfix({'status': 'Error', 'ID': i})
            raise e 
        
        # Validation and test with JFLEG
        if extra is not None:
            best_correction, t5sim, bert_score = evaluate_jfleg(sentence, extra, model_evaluation, tokenizer)
            mean_evaluation_score += t5sim
            
            with open(verification, 'a', encoding='utf-8') as file:
                file.write(best_correction[0] + '\n')
            
            if bert_score is not None:
                for key, value in bert_score.items():
                    val = value[0]
                    if key not in mean_bert:
                        mean_bert[key] = val
                    else:
                        mean_bert[key] += val
        else:
            continue
    
    mean_evaluation_score /= len(dataloader)
    result_test = f'Mean evaluation score: {mean_evaluation_score}\n'
    
    for key, value in mean_bert.items():
        mean_bert[key] /= len(dataloader)
        result_test += f'Mean {key} score: {mean_bert[key]}\n'
    
    return result if result != "" else result_test
        


In [33]:
# GEC(model='T5', output_folder='output', dataset='ASR')

Number of samples: 21

    Transcript n°0 :
    
    Transcript : She don't like to eat vegetables.
    


  0%|          | 0/21 [00:00<?, ?it/s]

Generating corrections for ASR with T5 in output/T5_ASR


12/06/2023 23:58:04 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


TypeError: HappyTextToText.eval() missing 1 required positional argument: 'input_filepath'

In [19]:
# GEC(model='T5', output_folder='output', dataset='jfleg')

Number of samples: 748

    Transcript n°0 :

    Path audio: data\
    Sentence: New and new technology has been introduced to the society .
    Corrections: ['New technology has been introduced to society .', 'New technology has been introduced into the society .', 'Newer and newer technology has been introduced into society .', 'Newer and newer technology has been introduced to the society .']
    


  0%|          | 0/748 [00:00<?, ?it/s]

Generating corrections for jfleg with T5 in output/T5_jfleg


12/11/2023 23:59:26 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
12/11/2023 23:59:33 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0
12/11/2023 23:59:33 - INFO - happytransformer.happy_transformer -   Initializing a pipeline


Mean evaluation score: 4.856417112299462
Mean precision score: 0.7269314780114169
Mean recall score: 0.7353363115201021
Mean f1 score: 0.7309015628328935


https://aclanthology.org/P17-1074.pdf

___
## Autres tests

Jfleg training:
https://huggingface.co/vennify/t5-base-grammar-correction


In [7]:
# model_b = T5ForConditionalGeneration.from_pretrained('t5-small').to("cuda")
# stsb_sentence_1 = preprocess_text
# stsb_sentence_2 = output
# input_ids = tokenizer.encode("stsb sentence 1: "+stsb_sentence_1+" sentence 2: "+stsb_sentence_2, return_tensors="pt").to("cuda")
# stsb_ids = model_b.generate(input_ids)
# stsb = tokenizer.decode(stsb_ids[0],skip_special_tokens=True)
# print(stsb)


5.0
